In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pysindy as ps
import re

# ── 1. 파일명만으로 데이터 로드 & 전처리 ──
file_names = [
    "RHa_0_RHc_100_0.csv",
    "RHa_30_RHc_80_0.csv",
    "RHa_80_RHc_80_0.csv",
]
dfs = []

for fn in file_names:
    # 여러 인코딩 시도
    for enc in ("utf-8", "cp949", "latin1"):
        try:
            df = pd.read_csv(fn, encoding=enc)
            break
        except UnicodeDecodeError:
            continue
    else:
        raise UnicodeDecodeError(f"모든 인코딩 시도가 실패했습니다: {fn}")

    df.rename(columns={"Voltage(V)": "V", "Current(A)": "I"}, inplace=True)
    df["J"] = df["I"] / 25.0
    m = re.match(r"RHa_(\d+)_RHc_(\d+)_\d+\.csv$", fn)
    df["RHa"]   = float(m.group(1))
    df["RHc"]   = float(m.group(2))
    df["t_orig"]= np.arange(len(df))
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

# ── 2. 시간 정규화 ──
t_max = df_all["t_orig"].max()
df_all["t"] = df_all["t_orig"] / t_max

# ── 3. 상태·도함수·제어 입력 준비 ──
x      = df_all["V"].values.reshape(-1,1)
t_orig = df_all["t_orig"].values
dt     = np.gradient(t_orig)
x_dot  = np.gradient(x.flatten(), t_orig).reshape(-1,1)
u      = df_all[["J","RHa","RHc","t"]].values

# ── 4. SINDy 모델 정의 & 학습 ──
library   = ps.PolynomialLibrary(degree=3, include_interaction=True)
optimizer = ps.STLSQ(threshold=0.1)

# control_names 대신 제거
model = ps.SINDy(
    feature_library=library,
    optimizer=optimizer,
    feature_names=["V"]
)

# u=u 로 제어 입력 전달
model.fit(x, t=dt, x_dot=x_dot, u=u)
model.print()

# ── 5. RHa=30, RHc=80, J=0~1.4(0.2 간격), V₀=0.96 로 예측 ──
J_vals      = np.arange(0, 1.41, 0.2)
t_pred_orig = np.linspace(0, t_max, 100)
t_pred_norm = t_pred_orig / t_max

V_end = []
for J in J_vals:
    u_pred = np.column_stack([
        np.full_like(t_pred_norm, J),
        np.full_like(t_pred_norm, 30.0),
        np.full_like(t_pred_norm, 80.0),
        t_pred_norm
    ])
    x0    = [0.96]
    x_sim = model.simulate(x0, t_pred_orig, u=u_pred)
    V_end.append(x_sim[-1])

# ── 6. 예측 IV 커브 그리기 ──
plt.figure(figsize=(8,6))
plt.plot(J_vals, V_end, 'o-', lw=2)
plt.xlabel("Current Density J (A/cm²)")
plt.ylabel("Voltage V (V)")
plt.title("SINDy Predicted IV Curve (RHa=30, RHc=80)")
plt.grid(True)
plt.tight_layout()
plt.show()


ValueError: Values in t should be in strictly increasing order.

In [ ]:
# ── 7. 추가: 학습된 SINDy로 RHa=30, RHc=80, J=0~1.4(0.2 간격) 예측 ──

import numpy as np
import matplotlib.pyplot as plt

# J 값 범위
J_vals = np.arange(0, 1.41, 0.2)

# 원본 최대 시간 (t_orig) 가져오기
t_max = df_all["t_orig"].max()

# 시뮬레이션에 사용할 시간 그리드 (원본 스케일)
t_pred_orig = np.linspace(0, t_max, 100)

# 정규화된 시간 그리드
t_pred_norm = t_pred_orig / t_max

V_end = []
for J in J_vals:
    # 제어 입력 배열: [J, RHa=30, RHc=80, t(정규화)]
    u_pred = np.column_stack([
        np.full_like(t_pred_norm, J),
        np.full_like(t_pred_norm, 30.0),
        np.full_like(t_pred_norm, 80.0),
        t_pred_norm
    ])
    # 초기 전압
    x0 = [0.96]
    # 시뮬레이션
    x_sim_pred = model.simulate(x0, t_pred_orig, u_pred)
    # 최종 시점 전압 저장
    V_end.append(x_sim_pred[-1])

# IV 커브(예측) 그리기
plt.figure(figsize=(8, 6))
plt.plot(J_vals, V_end, "o-", label="Predicted IV (end-of-run)")
plt.xlabel("Current Density J (A/cm²)")
plt.ylabel("Voltage V (V)")
plt.title("SINDy Predicted IV Curve (RHa=30, RHc=80)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
